## Prediction of Density of States (DOS) using Partial Radial Distribution Function (PRDF) 

We want to study the accuracy and time performance of the featurizations used in [Schutt et al paper](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.89.205118). Here in part 1, we load some inorganic crystal compounds with maximum 6 atoms per unit cell and compute the features. 

### Importing packages 

In [1]:
import numpy as np
import pandas as pd
import pymatgen as pmg

from pymatgen.core.molecular_orbitals import MolecularOrbitals
from pymatgen import MPRester

from matminer.data_retrieval.retrieve_MP import MPDataRetrieval
from matminer.utils.conversions import dict_to_object, str_to_composition
from matminer.featurizers.composition import AtomicOrbitals
from matminer.featurizers.structure import PartialRadialDistributionFunction 

from matminer.utils.data_files.deml_elementdata import atom_num

/home/amoffet2/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/amoffet2/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88

/home/amoffet2/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88



### Loading dataset 

In [2]:
mp = MPDataRetrieval(api_key='T6QzrvW8J07u4L2O')

Getting all dataset with less than 6 atoms per unit primitive cell

In [3]:
%%time
data = mp.get_dataframe(criteria={"nsites": {"$lte": 6}},
                        properties=["pretty_formula", "structure"])
print ("Shape of retrieved data: ", data.shape)

Shape of retrieved data:  (16590, 2)
CPU times: user 1.5 s, sys: 101 ms, total: 1.6 s
Wall time: 14.5 s


In [4]:
data.head(1)

,pretty_formula,structure
material_id,,
mp-85,In,"{'@module': 'pymatgen.core.structure', '@class..."


In [5]:
data.reset_index(inplace=True)

Convert structure to pymatgen structure object

In [7]:
data['structure_obj'] = dict_to_object(data['structure'])

Convert formula to pymatgen composition object

In [8]:
data['composition_obj'] = str_to_composition(data['pretty_formula'])

Compute orbitals occupied and set f orbital compounds to NaN.

In [9]:
data['max_atom_num'] = data['composition_obj'].apply(lambda x: max(atom_num[str(i)] for i in x))

In [10]:
def orbital_partition(x):
    if (x <= 20):
        return 'sp'
    elif (x > 20 and x < 70):
        return 'spd'
    else:
        return np.nan
    
data['max_orbital'] = data['max_atom_num'].apply(orbital_partition)

Drop compounds with f orbital

In [11]:
data.dropna(subset=['max_orbital'], inplace=True)

In [12]:
print ("Shape of data: ", data.shape)

Shape of data:  (7684, 7)


Get DOS data of materials using MPRester

In [13]:
%%time
mprester = MPRester(api_key='T6QzrvW8J07u4L2O')
def get_dos(id):
    try:
        return mprester.get_dos_by_material_id(id)
    except:
        return np.nan

data['dos_obj'] = data['material_id'].apply(get_dos)

CPU times: user 2min 24s, sys: 5.71 s, total: 2min 30s
Wall time: 1h 10min 47s


Drop data without Complete DOS value

In [14]:
data = data.dropna(subset=['dos_obj']).reset_index(drop=True)
print ("Shape of data: ", data.shape)

Shape of data:  (4829, 8)


Compute DOS at Fermi level using matminer DOSFeaturizer

In [15]:
def compute_dos(dos):
    try:
        total_density = sum(dos.densities.values()) #sum over both spins, if present
        min_index = np.argmin(abs(dos.energies - dos.efermi))
        return total_density[min_index] # returns states/eV/_unit_cell_
    except:
        return np.nan

In [16]:
data['dos'] = data['dos_obj'].apply(compute_dos)

`compute_dos` returns DOS in unit $states/eV/unit\_cell$. Here, we divide the DOS by volume of its structure to get $states/eV/A^3$.

In [17]:
data['volume'] = data['structure_obj'].apply(lambda x: x.volume)
data['dos'] = np.true_divide(data['dos'], data['volume'])

### Compute representation 

In [18]:
cutoff, bin_size = 10.0, 2.0
prdf = PartialRadialDistributionFunction(cutoff=cutoff, bin_size=bin_size)

In [19]:
prdf.fit(data['structure_obj'].tolist())

/home/atan14/.local/lib/python3.6/site-packages/pymatgen/core/periodic_table.py:409: UserWarning:

No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.

/home/atan14/.local/lib/python3.6/site-packages/pymatgen/core/periodic_table.py:409: UserWarning:

No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.

/home/atan14/.local/lib/python3.6/site-packages/pymatgen/core/periodic_table.py:409: UserWarning:

No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.



PartialRadialDistributionFunction(bin_size=1.0, cutoff=16.0, exclude_elems=[],
                 include_elems=[])

In [20]:
%%time
data = prdf.featurize_dataframe(data, col_id='structure_obj', ignore_errors=True)

/home/atan14/.local/lib/python3.6/site-packages/pymatgen/core/periodic_table.py:409: UserWarning:

No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.

/home/atan14/.local/lib/python3.6/site-packages/pymatgen/core/periodic_table.py:409: UserWarning:

No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.

/home/atan14/.local/lib/python3.6/site-packages/pymatgen/core/periodic_table.py:409: UserWarning:

No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.



CPU times: user 4min 21s, sys: 2min 6s, total: 6min 27s
Wall time: 8min 5s


#### Save featurized data as pickle file

In [21]:
data.to_pickle('./schutt_cutoff%s_binsize%s.pkl'%(cutoff, bin_size*10))